# Data Cleaning and Preprocessing
## House Prices in Grand Tunis - Data Mining Project

This notebook contains the data cleaning and preprocessing steps including filtering, outlier removal, and duplicate handling.

## Import Required Libraries and Load Data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load the dataset
df = pd.read_csv('../data/raw/source_1/Property-Prices-in-Tunisia.csv')
print(f"Initial dataset shape: {df.shape}")
display(df.head())

## Data Filtering - Focus on Grand Tunis Apartments

In [ ]:
grand_tunis_regions = ['Tunis', 'Ariana', 'Ben arous', 'La manouba']
df = df[df['category'] == 'Appartements']
df = df[df['type'] == 'À Vendre']
df = df[df['city'].isin(grand_tunis_regions)]
df['price'] = df['price']/1000
df = df.drop(columns=['category', 'type', 'log_price'])

df = df[df['price'] <= 3000]
df = df[~((df['price'] <= 70)  & (df['size'] >= 70))]
df = df[~((df['price'] >= 1000)  & (df['size'] <= 90))]
df = df[~((df['room_count'] >= 2)  & (df['size'] <= 25))]

print(f"After filtering and outlier removal: {df.shape}")
display(df.head())

## Checking for Duplicate Rows

In [ ]:
# Check for exact duplicate rows
exact_duplicates = df[df.duplicated(keep=False)]

print(f"Number of exact duplicate rows: {len(exact_duplicates)}")

if not exact_duplicates.empty:
    print("Exact duplicate rows:")
    display(exact_duplicates.sort_values(by=list(df.columns)))
else:
    print("No exact duplicate rows found.")

## Removing Exact Duplicate Rows

In [ ]:
initial_rows = len(df)
df.drop_duplicates(inplace=True)
rows_after_deduplication = len(df)
print(f"Number of rows before removing exact duplicates: {initial_rows}")
print(f"Number of rows after removing exact duplicates: {rows_after_deduplication}")
print(f"Number of exact duplicates removed: {initial_rows - rows_after_deduplication}")

#### Checking for Partial Duplicates (based on identifying features)

We'll check for duplicates considering all features except `price`, as a property might be listed multiple times with slight price variations, or we might want to identify properties with identical characteristics.

In [ ]:
# Define columns to consider for partial duplicates (excluding 'price')
columns_for_partial_check = ['room_count', 'bathroom_count', 'size', 'city', 'region']

partial_duplicates = df[df.duplicated(subset=columns_for_partial_check, keep=False)]

print(f"Number of partial duplicate rows (based on {', '.join(columns_for_partial_check)}): {len(partial_duplicates)}")

if not partial_duplicates.empty:
    print("Partial duplicate rows:")
    display(partial_duplicates.sort_values(by=columns_for_partial_check))
else:
    print("No partial duplicate rows found based on the selected criteria.")

## Price per Square Meter Analysis

In [ ]:
df['price_per_sqm'] = (df['price'] * 1000) / df['size']
df = df[df['price_per_sqm'] <= 6000 ]

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['price_per_sqm'], bins=50, kde=True)
plt.title('Distribution of Price per Square Meter')
plt.xlabel('Price per m² (TND)')
plt.ylabel('Frequency')
plt.axvline(df['price_per_sqm'].median(), color='red', linestyle='--', label=f"Median: {df['price_per_sqm'].median():.0f}")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
df = df.drop(columns=['price_per_sqm'])

# Save cleaned data for next notebook
df.to_csv('../data/processed/source_1/apartments_cleaned.csv', index=False)
print(f"Cleaned dataset saved. Final shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")